In [26]:
# This Python 3 environment comes with many helpful analytics libraries installed
# It is defined by the kaggle/python Docker image: https://github.com/kaggle/docker-python
# For example, here's several helpful packages to load

import numpy as np # linear algebra
import pandas as pd # data processing, CSV file I/O (e.g. pd.read_csv)

# Input data files are available in the read-only "../input/" directory
# For example, running this (by clicking run or pressing Shift+Enter) will list all files under the input directory

import os
for dirname, _, filenames in os.walk('/kaggle/input'):
    for filename in filenames:
        pass
        #print(os.path.join(dirname, filename))

# You can write up to 20GB to the current directory (/kaggle/working/) that gets preserved as output when you create a version using "Save & Run All" 
# You can also write temporary files to /kaggle/temp/, but they won't be saved outside of the current session

**Import Dependencies, Framework and Libraries**

In [27]:
import torch
import torchvision
import torch.nn as nn
import torch.nn.functional as F
from torch.utils.data import Dataset, dataloader
from torchvision import datasets

from pathlib import Path
from torch import optim
!pip install torchmetrics
from torchmetrics import Accuracy, Precision, F1Score

**Get Data of Agricultural Products **

In [28]:
dir_path = Path('../input/agricultural-crops-image-classification/Agricultural-crops')

In [29]:
##Transform Data
from torchvision import transforms
transformer = torchvision.transforms.Compose([
    transforms.Resize(size = (224, 224)),
    transforms.ToTensor(),
])

In [30]:
##Load Image to Dataset 
datafolder = torchvision.datasets.ImageFolder(dir_path, transform = transformer)
len(datafolder)

In [31]:
classes = datafolder.classes

In [32]:
# Load a Sample
img, label = datafolder[0]

In [33]:
# Sample Details
img.shape, label

In [34]:
import matplotlib.pyplot as plt
img_sample = img.permute(2, 1, 0)
print(img.shape)
plt.imshow(img_sample)
plt.title(classes[label]);

In [35]:
# set up device agnostic code
device = 'cuda' if torch.cuda.is_available() else 'cpu'

In [36]:
import torch.utils.data as data

# Random split
train_set_size = int(len(datafolder) * 0.8)
valid_set_size = len(datafolder) - train_set_size
train_set, valid_set = data.random_split(datafolder, [train_set_size, valid_set_size])

In [37]:
## lenghth of Sets
len(train_set), len(valid_set)

In [38]:
torch.__version__

In [39]:
import torchvision
from torchvision import models
import torchvision.models.efficientnet as e
import torchvision.models.resnet as r

In [40]:
#Load Weights for Training
WEIGHTS = r.resnet50(pretrained= True)
WEIGHTS2 = e.efficientnet_b0(pretrained= True)

In [41]:
WEIGHTS.state_dict

In [42]:
for p in WEIGHTS.parameters():
    p.requires_grad = False

In [43]:
WEIGHTS.fc = nn.Linear(2048, 30)

In [44]:
WEIGHTS.fc

In [45]:
#datafolder = torchvision.datasets.ImageFolder(dir_path, transform = weights_transforms)
#train_set, valid_set = data.random_split(datafolder, [train_set_size, valid_set_size])
from torch.utils.data import DataLoader
BATCH_SIZE = 32
train_loader = DataLoader(dataset =train_set, batch_size =BATCH_SIZE, shuffle = True)
val_loader = DataLoader(dataset= valid_set, batch_size = BATCH_SIZE)

##Evaluation Metrics/ Loss Function / Optimizers

In [46]:
model = WEIGHTS

In [47]:
model.to(device)

In [48]:
loss_fn = nn.CrossEntropyLoss().to(device)
optimizer = optim.Adam(model.parameters(), lr= 0.01)
accuracy_fn = Accuracy().to(device)

In [49]:
EPOCHS = 5

train_losses, train_acc = 0, 0

for epoch in range(EPOCHS):
  for batch, (X, y )in enumerate(train_loader):
    X = X.to(device)
    y = y.to(device)
    model.train()

    y_pred =model(X)
    loss = loss_fn(y_pred, y)
    train_acc += accuracy_fn(y_pred.argmax(dim=1), y)
    train_losses += loss
    optimizer.zero_grad()
    loss.backward()
    optimizer.step()
    
    if batch % 100 == 0:
      print(f'{train_losses} | {train_acc}')

  train_losses /= len(train_loader)
  train_acc /= len(train_loader)
  print(f' Train Loss: {train_losses:.4f} | Train Acc: {train_acc:.3f}')

model.eval()
test_loss, test_acc = 0, 0
with torch.inference_mode():
    for Xt, yt in val_loader:
        Xt = Xt.to(device)
        yt= yt.to(device)
        test_pred = model(X)
        t_loss = loss_fn(test_pred, y)
        test_loss += t_loss
        test_acc += accuracy_fn(test_pred.argmax(dim=1), y)
    test_loss /= len(val_loader)
    test_acc /= len(val_loader)
    
    print(f' Test Loss {test_loss:.4f} | Test Acc: {test_acc:.2f}   ')
